In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### LangGraph 주요 컴포넌트

```
스테이트: 그래프의 상태 표현
```

In [2]:
import operator
from typing import Annotated
from langchain_core.pydantic_v1 import BaseModel, Field

class State(BaseModel):
  query: str = Field(..., description='사용자의 질문')
  current_role: str = Field(default='', description='선정된 답변 역할')
  messages: Annotated[list[str], operator.add] = Field(defualt=[], description='답변 이력')
  current_judge: bool = Field(default=False, description='품질 검사 결과')
  judgement_reason: str = Field(deafult='', description='품질 검사 판정 이유')

c:\Users\user\anaconda3\envs\skn\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

```
노드: 그래프를 구성하는 처리 단위
```
노드의 지정 방법

In [ ]:
# 함수 또는 Runnable만 지정하는 예
# 이 경우 노드 이름 'answering_node'가 됨
# 방법1 : workflow.add_node(answering_node)

# answering_node이라는 이름의 노드를 정의
# 노드 내 처리는 answering_ndoe 함수가 수행
# 방법2 : workflow.add_node('answering_node', answering_node)

노드의 구현 방법

In [6]:
from typing import Any

def answering_node(state: State) -> dict[str, Any]:
  query = state.query
  role = state.current_role

  # 사용자 질문 내용과 선택된 역할을 바탕으로 답변을 생성하는 로직
  generated_message = "...생성 처리..."

  # 생성된 답변으로 스테이트 업데이트
  return {'messages': [generated_message]}

def check_node(state: State) -> dict[str, Any]:
  query = state.query
  message = state.messages[-1]

  # 사용자의 질문 내용과 답변 내용에서 품질 검사를 수행하는 처리
  judge = "...판정 결과..."
  reason = "...판정 이유..."

  # 생성된 답변으로 스테이트 업데이트
  return {"current_judge": judge, 'judgement_reason': reason}

```
엣지: 노드 간의 연결
```

엔트리 포인트

In [ ]:
# 그래프 시작 노드 지정 엣지

workflow.set_entry_point('selection')
workflow.add_edge('START', 'selection')

엣지

In [ ]:
# 특정 노드에서 다른 노드로 무조건 전이하는 엣지
workflow.add_edge('selection', 'answering')

조건부 엣지

In [ ]:
# 조건에 기반하여 전이할 노드를 결정하는 엣지
from langgraph.graph import END

workflow.add_conditional_edges(
  'check',
  lambda state: state.current_judge,
  {True: END, False: 'selection'}
)

```
컴파일된 그래프
```

In [ ]:
# 정의된 그래프는 complie함수를 통해 실행 가능한 CompliedGraph 인스턴스로 변환
complied = workflow.compile()

invoke 함수

In [ ]:
# invoke 함수를 사용하면 그래프 내의 모든 처리가 실행된 후 최종 값이 반환
initial_state = State(query='생성형 AI에 관해 알려주세요')
result = complied.invoke(initial_state)

ainvoke 함수

In [ ]:
# 작동은 invoke 함수와 동일하지만, ainvoke 함수를 사용하면 비동기 함수로 실행할 수 있음
initial_state = State(query='생성 AI에 관해 알려주세요')
result = await complied.ainvoke(initial_state)

stream 함수

In [ ]:
# 노드 실행 시 스테이트를 순차적으로 가져올 수 있음
initial_state = State(query='생성 AI에 관해 알려주세요')
for step in compiled.stream(initial_state):
  print(step)